In [1]:
import pandas as pd
import zipcodes
import numpy as np
import datetime

# Demo by zip

In [20]:
# HH above>25
# HH meidan income
1+1

2

In [3]:
EASI_A_Income=pd.read_csv("/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_A2_CSV/ZIP4_18_DATA_A2.CSV",skiprows=1,
                 dtype=str,usecols=['ZIP_CODE','ZIP4','HH18','MEDHHINC'])

In [4]:
EASI_A_Income['HH18']=EASI_A_Income['HH18'].astype(float)
EASI_A_Income['MEDHHINC']=EASI_A_Income['MEDHHINC'].astype(float)

In [5]:
Median_Income_Zip5=pd.DataFrame()
for zip_cd,group in EASI_A_Income.groupby(['ZIP_CODE']):
    total_HH=group['HH18'].sum()
    if total_HH>0:
        group['weight']=group['HH18']/total_HH
        group['weighted_income']=group['weight']*group['MEDHHINC']
        weighted_income=group['weighted_income'].sum()
    else:
        weighted_income=0
    df=pd.DataFrame({"zip_cd":zip_cd,"Median_Income":weighted_income},index=[zip_cd])
    Median_Income_Zip5=Median_Income_Zip5.append(df)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
EASI_C_HH_25_74=pd.read_csv("/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_C2_CSV/ZIP4_18_DATA_C2.CSV",dtype=str,skiprows=1,
                            usecols=['ZIP_CODE','ZIP4','HH18','PCTHH25_34','PCTHH35_44','PCTHH45_54','PCTHH55_64','PCTHH65_74'])
for col in EASI_C_HH_25_74.columns.tolist()[2:]:
    EASI_C_HH_25_74[col]=EASI_C_HH_25_74[col].astype(float)
EASI_C_HH_25_74['HH_25_74_Pctg']=EASI_C_HH_25_74[EASI_C_HH_25_74.columns.tolist()[3:8]].sum(axis=1)/100
EASI_C_HH_25_74['HH_25_74']=EASI_C_HH_25_74['HH_25_74_Pctg']*EASI_C_HH_25_74['HH18']
HH_25_74_Zip5=EASI_C_HH_25_74.groupby(['ZIP_CODE'])['HH_25_74'].sum().to_frame().reset_index()
HH_25_74_Zip5=HH_25_74_Zip5.rename(columns={"ZIP_CODE":"zip_cd"})

In [7]:
demo_zip5=pd.merge(HH_25_74_Zip5,Median_Income_Zip5,on="zip_cd",how="outer")

In [8]:
Median_Income_Zip5.head(2)

,Median_Income,zip_cd
00501,0.000000,00501
01001,80804.813047,01001


# Zips Group

In [9]:
JUBA_zips=pd.read_excel("/home/jian/Projects/Saatva/Analysis/Zips_Comparison_Saatva_Juba/data/Saatva microsegment_083018_JC V2.xlsx",
                       dtype=str,sheetname="Zip code",usecols=[15,16,17,18],skiprows=1)
JUBA_zips_list=JUBA_zips['HH Income 100k+_1'].unique().tolist()+\
                JUBA_zips['HH Income 100k+_2'].unique().tolist()+\
                JUBA_zips['HH Income 75k-100k_1'].unique().tolist()+\
                JUBA_zips['HH Income 75k-100k_2'].unique().tolist()
JUBA_zips_list=set(JUBA_zips_list)
JUBA_zips_list.remove("nan")

In [10]:
Saatva_zips=pd.read_excel("/home/jian/Projects/Saatva/Analysis/Zips_Comparison_Saatva_Juba/data/Saatva Zipcodes from Saatva Geo Modifiers_forMS_9_5_2018.xlsx",dtype=str)
Saatva_zips['zips']=Saatva_zips['Location'].apply(lambda x: x.split(", ")[0])
Saatva_zips['zip_or_city']=Saatva_zips['zips'].apply(lambda x: x[0] in [str(i) for i in range(10)])

Saatva_zips_df_1=Saatva_zips[Saatva_zips['zip_or_city']==True]
Saatva_zips_list_1=Saatva_zips_df_1['zips'].unique().tolist()

Saatva_zips_df_2=Saatva_zips[Saatva_zips['zip_or_city']==False]
Saatva_zips_df_2=Saatva_zips_df_2[Saatva_zips_df_2['Location']!="United States"].reset_index()
Saatva_zips_df_2=Saatva_zips_df_2.rename(columns={"zips":"cities"})
Saatva_zips_df_2['states']=Saatva_zips_df_2['Location'].apply(lambda x: x.split(", ")[1])
Saatva_zips_df_2['cities']=Saatva_zips_df_2['cities'].apply(lambda x: x.lower())

In [11]:
state_abb=pd.read_csv("/home/jian/Docs/states.csv",dtype=str)
state_abb_dict=state_abb.set_index("State").to_dict()['Abbreviation']
Saatva_zips_df_2['states']=Saatva_zips_df_2['states'].apply(lambda x: state_abb_dict[x])
Saatva_zips_df_2['City_State']=Saatva_zips_df_2['cities']+", "+Saatva_zips_df_2['states']

In [12]:
zip_city=pd.read_csv("/home/jian/Docs/Geo_mapping/free-zipcode-database.csv",dtype=str)
zip_city['City']=zip_city['City'].apply(lambda x: x.lower())
zip_city['zip_cd']=zip_city['Zipcode'].apply(lambda x: x.zfill(5))
zip_city['City_State']=zip_city['City']+", "+zip_city['State']

Saatva_zips_df_2['place_in_cities']=Saatva_zips_df_2['cities'].apply(lambda x: x in zip_city['City'].tolist())
Saatva_zips_df_2_city_list=Saatva_zips_df_2[Saatva_zips_df_2['place_in_cities']==True]['City_State'].unique().tolist()

zips_in_cities=zip_city[zip_city['City_State'].isin(Saatva_zips_df_2_city_list)]
Saatva_zips_list_2=zips_in_cities['zip_cd'].unique().tolist()

In [13]:
len(zips_in_cities['City_State'].unique())

31

In [14]:
Saatva_zips_total_list=set(Saatva_zips_list_1+Saatva_zips_list_2)

In [15]:
Seg_1_Juba_Only=[x for x in JUBA_zips_list if x not in Saatva_zips_total_list]
Seg_2_Saatva_Only=[x for x in Saatva_zips_total_list if x not in JUBA_zips_list]
Seg_3_Both=[x for x in Saatva_zips_total_list if x in JUBA_zips_list]

# output

In [16]:
demo_zip5['Group']=np.where(demo_zip5['zip_cd'].isin(Seg_1_Juba_Only),"G1_Juba_Only",
                           np.where(demo_zip5['zip_cd'].isin(Seg_2_Saatva_Only),"G2_Saatva_Only",
                                   np.where(demo_zip5['zip_cd'].isin(Seg_3_Both),"G3_Overlap",
                                            np.where(demo_zip5['Median_Income']>=75000,"Neither_75_Above","Neither_Lower_Balance"
                                                    )
                                           )
                                   )
                           )
filter_output=demo_zip5.copy()
filter_output=filter_output[filter_output['HH_25_74']>=50]

In [17]:
writer=pd.ExcelWriter("/home/jian/Projects/Saatva/Analysis/Zips_Comparison_Saatva_Juba/output/Saatva_zips_for_FB_grouped_by_srouce_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
filter_output['score']=filter_output['HH_25_74']*filter_output['Median_Income']
output=pd.DataFrame()
for Group,group in filter_output.groupby(['Group']):
    group=group.sort_values(['score'],ascending=False).reset_index()
    del group ['index']
    group['rank_in_group']=[x for x in range(len(group))]
    total_rows=len(group)
    if total_rows>2499:
        half_total=total_rows/2
        group['inner_seg_2500']=group['rank_in_group'].apply(lambda x: str(int(np.floor(x/half_total))+1))
    else:
        group['inner_seg_2500']="1"
    group['Group']=group['Group']+"_"+group['inner_seg_2500']
    group=group[['Group','zip_cd','HH_25_74','Median_Income','score']]
    output=output.append(group)
    group.to_excel(writer,Group,index=False)


In [18]:
neither_random_2499=filter_output[filter_output['Group']=="Neither_Lower_Balance"].reset_index()
del neither_random_2499['index']
neither_random_2499=neither_random_2499.reset_index()

import random
random.seed(a=1)
chosen_rows=random.sample(range(1, len(neither_random_2499)), 2499)
neither_random_2499=neither_random_2499[neither_random_2499['index'].isin(chosen_rows)]
del neither_random_2499['index']
neither_random_2499.to_excel(writer,"Random_2499_Balance_Neither",index=False)

In [19]:
output=output[['Group','zip_cd','HH_25_74','Median_Income','score']]
output.to_excel(writer,'All_Zips',index=False)
writer.save()